# Boilerplate notebook

In [39]:
# Matplotlib
import matplotlib.pyplot as plt
# Numpy
import numpy as np
# Pillow
from PIL import Image
# Torch
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
from torchvision import transforms
from torchsummary import summary
# Misc
import time
from datetime import datetime

# 1. Download dataset

In [2]:
!git clone -b data https://github.com/Oxiang/50.039-Deep-Learning.git

fatal: destination path '50.039-Deep-Learning' already exists and is not an empty directory.


In [3]:
!sudo apt-get install tree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tree is already the newest version (1.7.0-5).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [2]:
cd 50.039-Deep-Learning

C:\Users\sceps\OneDrive\Personal Folder\SUTD Deep Learning\dl boiler\50.039-Deep-Learning\notebooks\colab\50.039-Deep-Learning


In [5]:
%%bash

(
tree dataset -d
) 

dataset
├── test
│   ├── infected
│   │   ├── covid
│   │   └── non-covid
│   └── normal
├── train
│   ├── infected
│   │   ├── covid
│   │   └── non-covid
│   └── normal
└── val
    ├── infected
    │   ├── covid
    │   └── non-covid
    └── normal

15 directories


In [3]:
classes_n_c = {0: 'normal', 1: 'infected'}
classes_inc_ic = {0: 'infected_non_covid', 1: 'infected_covid'}
groups = ['train', 'test', 'val']
dataset_numbers = {
    'train_normal': 1341,
    'train_infected_non_covid': 2530,
    'train_infected_covid': 1345,
    'val_normal': 8,
    'val_infected_non_covid': 8,
    'val_infected_covid': 8,    
    'test_normal': 234,
    'test_infected_non_covid': 242,
    'test_infected_covid': 138,
}
dataset_paths = {
    'train_normal': './dataset/train/normal/',
    'train_infected_non_covid': './dataset/train/infected/non-covid/',
    'train_infected_covid': './dataset/train/infected/covid/',
    'val_normal': './dataset/val/normal/',
    'val_infected_non_covid': './dataset/val/infected/non-covid/',
    'val_infected_covid': './dataset/val/infected/covid/',    
    'test_normal': './dataset/test/normal/',
    'test_infected_non_covid': './dataset/test/infected/non-covid/',
    'test_infected_covid': './dataset/test/infected/covid/',    
}

View one of the images and its properties. These images consist of a Numpy array, with values ranging between 0 and 255. These values will be normalized.

# 2. Creating a Dataset object

## 2.1 Common variables

In [5]:
binary_dataset_paths = {
    'layer_0': {
        'train': {
            'train_normal':'./dataset/train/normal',
            'train_infected': './dataset/train/infected'
        },
        'val': {
            'val_normal':'./dataset/val/normal',
            'val_infected': './dataset/val/infected'
        },
        'test': {
            'test_normal':'./dataset/test/normal',
            'test_infected': './dataset/test/infected'
        }
    },
    'layer_1':{
        'train': {
            'train_covid': './dataset/train/infected/covid',
            'train_non_covid' : './dataset/train/infected/non-covid'
        },
        'val': {
            'val_covid': './dataset/val/infected/covid',
            'val_non_covid' : './dataset/val/infected/non-covid'
        },
        'test': {
            'test_covid': './dataset/test/infected/covid',
            'test_non_covid' : './dataset/test/infected/non-covid'            
        }
    }
}

binary_dataset_numbers = {
    'layer_0': {
        'train': {
            'train_normal': 1341,
            'train_infected': 3875
        },
        'val': {
            'val_normal': 8,
            'val_infected': 16
        },
        'test': {
            'test_normal': 234,
            'test_infected': 380
        }
    },
    'layer_1':{
        'train': {
            'train_covid': 1345,
            'train_non_covid' : 2530
        },
        'val': {
            'val_covid': 8,
            'val_non_covid': 8
        },
        'test': {
            'test_covid': 138,
            'test_non_covid': 242            
        }
    }
}

## 2.2 Layer 0 General Dataset object that is custom made for train, val, test to individually use

length method ( __ len __ )

> return the number of images present in the dataset

getitem method ( __ getitem __ )

> fetch an image and its label, using a single index value. Returns the image, along with a one-hot vector corresponding to the class of the object. Both returned parameters will be torch tensors.
- [1, 0] for normal class
- [0, 1] for infected class

In [6]:
class L0_Lung_Dataset(Dataset):
    """
    Generic Dataset class for Layer 0
    """
    
    def __init__(self, groups, dataset_numbers, dataset_paths, infected_sub_class_numbers):
        """
        Constructor for generic Dataset class for Layer0 - assembles
        the important parameters in attributes.

        Parameters
        ----------
        groups : str
            Allowed values: train, val, test
        dataset_numbers : dict
            Count of each class within specified group (e.g. normal, infected)
        dataset_paths : dict
            Path to each class within specified group (infected has 2 sub-class dir)
        """

        self.img_size = (150, 150)
        self.classes = { 0: 'normal', 1: 'infected' }
        self.covid_status = {0: '', 1: 'covid', 2: 'non-covid'} 
        self.groups = groups
        self.dataset_numbers = dataset_numbers
        self.dataset_paths = dataset_paths
        self.infected_sub_class_numbers = infected_sub_class_numbers

        
    def describe(self):
        """
        Descriptor function.
        Will print details about the dataset when called.
        """
        
        # Generate description
        msg = "This is the {} dataset of the Lung Dataset".format(self.groups)
        msg += " used for the Small Project Demo in the 50.039 Deep Learning class"
        msg += " in March 2021. \n"
        msg += "It contains a total of {} images, ".format(sum(self.dataset_numbers.values()))
        msg += "of size {} by {}.\n".format(self.img_size[0], self.img_size[1])
        msg += "The images are stored in the following locations "
        msg += "and each one contains the following number of images:\n"
        for key, val in self.dataset_paths.items():
            msg += " - {}, in folder {}: {} images.\n".format(key, val, self.dataset_numbers[key])
        print(msg)
        
    
    def open_img(self, group_val, class_val, covid_status, index_val):
        """
        Opens image with specified parameters.
        
        Parameters:
        - group_val should take values in 'train', 'test' or 'val'.
        - class_val variable should be set to 'normal' or 'infected_non_covid' or 'infected_covid'.
        - covid_status should take values in '', 'covid' or 'non_covid'.
        - index_val should be an integer with values between 0 and the maximal number of images in dataset.
        
        Returns loaded image as a normalized Numpy array.
        """
        
        # Asserts checking for consistency in passed parameters
        err_msg = "Error - group_val variable should be set to 'train', 'test' or 'val'."
        assert group_val in self.groups, err_msg
        
        err_msg = "Error - class_val variable should be set to 'normal' or 'infected_non_covid' or 'infected_covid."
        assert class_val in self.classes.values(), err_msg

        err_msg = "Error - covid_status variable should be set to '', 'covid' or 'non-covid'."
        assert covid_status in self.covid_status.values(), err_msg
        
        max_val = self.dataset_numbers['{}_{}'.format(group_val, class_val)]
        err_msg = "Error - index_val variable should be an integer between 0 and the maximal number of images."
        err_msg += "\n(In {}/{}, you have {} images.)".format(group_val, class_val, max_val)
        assert isinstance(index_val, int), err_msg
        assert index_val >= 0 and index_val <= max_val, err_msg
        
        # 'normal' - example path: /dataset/train/normal/1.jpg
        if covid_status == "":
            path_to_file = '{}/{}.jpg'.format(self.dataset_paths['{}_{}'.format(group_val, class_val)], index_val)
        # 'covid' or 'non_covid' - example path: './dataset/train/infected/covid/1.jpg',
        else:
            path_to_file = '{}/{}/{}.jpg'.format(self.dataset_paths['{}_{}'.format(group_val, class_val)], covid_status, index_val)

        with open(path_to_file, 'rb') as f:
            # Convert to Numpy array and normalize pixel values by dividing by 255.
            im = np.asarray(Image.open(f))/255
        f.close()
        return im
    
    
    def show_img(self, group_val, class_val, covid_status, index_val):
        """
        Opens, then displays image with specified parameters.
        
        Parameters:
        - group_val should take values in 'train', 'test' or 'val'.
        - class_val variable should be set to 'normal' or 'infected'.
        - covid_status should take values in '', 'covid' or 'non-covid'.
        - index_val should be an integer with values between 0 and the maximal number of images in dataset.
        """
        
        # Open image
        im = self.open_img(group_val, class_val, covid_status, index_val)
        
        # Display
        plt.imshow(im)

    def __len__(self):
        """
        Length special method, returns the number of images in dataset.
        """
        
        # Length function
        return sum(self.dataset_numbers.values())
    
    
    def __getitem__(self, index):
        """
        Getitem special method.
        
        Expects an integer value index, between 0 and len(self) - 1.
        
        Returns the image and its label as a one hot vector, both
        in torch tensor format in dataset.
        """
        
        # Get item special method
        first_val = int(list(self.dataset_numbers.values())[0])
        if index < first_val:
            class_val = 'normal'
            label = torch.Tensor([1, 0])
            covid_status = ""
        else:
            class_val = 'infected'
            index = index - first_val
            label = torch.Tensor([0, 1])
            infected_covid_numbers = int(list(self.infected_sub_class_numbers.values())[0]) # covid
            if index < infected_covid_numbers:
                class_val = 'infected'
                covid_status = 'covid'
            else:
                class_val = 'infected'
                index = index - infected_covid_numbers
                covid_status = 'non-covid'
        im = self.open_img(self.groups, class_val, covid_status, index)
        im = transforms.functional.to_tensor(np.array(im)).float()
        return im, label

In [7]:
def verify_l0_dataset(group,dataset,image_overall_index,class_val,covid_status,
                   image_specific_dataset_index=1):
  """
  Helper function to verify that the classes are implemented correctly

  Parameters
  ----------
  group : str
      Allowed values: train, val, test
  dataset: object
      Object instantiated from the class
  image_overall_index: int
      Overall index in the full dataset across all classes
  class_val: str
      Image label. Example: normal, infected
  covid_status: str
      If class_val is 'infected', set this to either 'covid' or 'non-covid'
  image_specific_dataset_index : int
      image id in the specific nested directory
  """
  print('Verify the special methods __len__ and __get_item__')
  print('Number of images in {} dataset: {}'.format(group, len(dataset)))
  print('Details for image id {} from the {} dataset'.format(
      image_overall_index,
      group
  ))
  im, class_oh = dataset[image_overall_index]
  print('Sample image shape: {}'.format(im.shape))
  print('Sample image: {}'.format(im))
  print('Sample image class: {}'.format(class_oh))

  print('\nVerify the open_img and show_img functions')
  print('Open and show image {} from the {}_{} dataset'.format(
      image_specific_dataset_index,
      group,
      class_val
  ))
  im = dataset.open_img(group, class_val, covid_status, image_specific_dataset_index)
  print('Sample image shape: {}'.format(im.shape))
  print('Sample image: {}'.format(im))
  dataset.show_img(group, class_val, covid_status, image_specific_dataset_index)

### 2.2.1 Layer 0 Train dataset

In [8]:
train_group = 'train'
l0_ld_train = L0_Lung_Dataset(groups = train_group,
                              dataset_numbers = binary_dataset_numbers['layer_0'][train_group],
                              dataset_paths = binary_dataset_paths['layer_0'][train_group],
                              infected_sub_class_numbers = binary_dataset_numbers['layer_1'][train_group])
l0_ld_train.describe()

This is the train dataset of the Lung Dataset used for the Small Project Demo in the 50.039 Deep Learning class in March 2021. 
It contains a total of 5216 images, of size 150 by 150.
The images are stored in the following locations and each one contains the following number of images:
 - train_normal, in folder ./dataset/train/normal: 1341 images.
 - train_infected, in folder ./dataset/train/infected: 3875 images.



### 2.2.2 Layer 0 Validation dataset

In [10]:
val_group = 'val'
l0_ld_val = L0_Lung_Dataset(groups = val_group,
                            dataset_numbers = binary_dataset_numbers['layer_0'][val_group],
                            dataset_paths = binary_dataset_paths['layer_0'][val_group],
                            infected_sub_class_numbers = binary_dataset_numbers['layer_1'][val_group])
l0_ld_val.describe()

This is the val dataset of the Lung Dataset used for the Small Project Demo in the 50.039 Deep Learning class in March 2021. 
It contains a total of 24 images, of size 150 by 150.
The images are stored in the following locations and each one contains the following number of images:
 - val_normal, in folder ./dataset/val/normal: 8 images.
 - val_infected, in folder ./dataset/val/infected: 16 images.



### 2.2.3 Layer 0 Test dataset

In [12]:
test_group = 'test'
l0_ld_test = L0_Lung_Dataset(groups = test_group, 
                              dataset_numbers = binary_dataset_numbers['layer_0'][test_group], 
                              dataset_paths = binary_dataset_paths['layer_0'][test_group],
                              infected_sub_class_numbers = binary_dataset_numbers['layer_1'][test_group])
l0_ld_test.describe()

This is the test dataset of the Lung Dataset used for the Small Project Demo in the 50.039 Deep Learning class in March 2021. 
It contains a total of 614 images, of size 150 by 150.
The images are stored in the following locations and each one contains the following number of images:
 - test_normal, in folder ./dataset/test/normal: 234 images.
 - test_infected, in folder ./dataset/test/infected: 380 images.



## 2.3 Layer 1 General Dataset object that is custom made for train, val, test to individually use

In [14]:
class L1_Lung_Dataset(Dataset):
    """
    Generic Dataset class for Layer 1
    """
    
    def __init__(self, groups, dataset_numbers, dataset_paths):
        """
        Constructor for generic Dataset class for Layer0 - assembles
        the important parameters in attributes.

        Parameters
        ----------
        groups : str
            Allowed values: train, val, test
        dataset_numbers : dict
            Count of each class within specified group (e.g. covid, non_covid)
        dataset_paths : dict
            Path to each class within specified group (infected has 2 sub-class dir)
        """

        self.img_size = (150, 150)
        self.classes = {0: 'covid', 1: 'non_covid'}
        self.groups = groups
        self.dataset_numbers = dataset_numbers
        self.dataset_paths = dataset_paths

        
    def describe(self):
        """
        Descriptor function.
        Will print details about the dataset when called.
        """
        
        # Generate description
        msg = "This is the {} dataset of the Lung Dataset".format(self.groups)
        msg += " used for the Small Project Demo in the 50.039 Deep Learning class"
        msg += " in March 2021. \n"
        msg += "It contains a total of {} images, ".format(sum(self.dataset_numbers.values()))
        msg += "of size {} by {}.\n".format(self.img_size[0], self.img_size[1])
        msg += "The images are stored in the following locations "
        msg += "and each one contains the following number of images:\n"
        for key, val in self.dataset_paths.items():
            msg += " - {}, in folder {}: {} images.\n".format(key, val, self.dataset_numbers[key])
        print(msg)
        
    
    def open_img(self, group_val, class_val, index_val):
        """
        Opens image with specified parameters.
        
        Parameters:
        - group_val should take values in 'train', 'test' or 'val'.
        - class_val variable should be set to 'covid' or 'non-covid'.
        - index_val should be an integer with values between 0 and the maximal number of images in dataset.
        
        Returns loaded image as a normalized Numpy array.
        """
        
        # Asserts checking for consistency in passed parameters
        err_msg = "Error - group_val variable should be set to 'train', 'test' or 'val'."
        assert group_val in self.groups, err_msg
        
        err_msg = "Error - class_val variable should be set to 'covid' or 'non-covid'."
        assert class_val in self.classes.values(), err_msg     
        
        max_val = self.dataset_numbers['{}_{}'.format(group_val, class_val)]
        err_msg = "Error - index_val variable should be an integer between 0 and the maximal number of images."
        err_msg += "\n(In {}/{}, you have {} images.)".format(group_val, class_val, max_val)
        assert isinstance(index_val, int), err_msg
        assert index_val >= 0 and index_val <= max_val, err_msg
        
        # Open file as before
        path_to_file = '{}/{}.jpg'.format(self.dataset_paths['{}_{}'.format(group_val, class_val)], index_val)
        with open(path_to_file, 'rb') as f:
            im = np.asarray(Image.open(f))/255
        f.close()
        return im
    
    
    def show_img(self, group_val, class_val, index_val):
        """
        Opens, then displays image with specified parameters.
        
        Parameters:
        - group_val should take values in 'train', 'test' or 'val'.
        - class_val variable should be set to 'covid' or 'non-covid'.
        - index_val should be an integer with values between 0 and the maximal number of images in dataset.
        """
        
        # Open image
        im = self.open_img(group_val, class_val, index_val)
        
        # Display
        plt.imshow(im)
        
        
    def __len__(self):
        """
        Length special method, returns the number of images in dataset.
        """
        
        # Length function
        return sum(self.dataset_numbers.values())
    
    
    def __getitem__(self, index):
        """
        Getitem special method.
        
        Expects an integer value index, between 0 and len(self) - 1.
        
        Returns the image and its label as a one hot vector, both
        in torch tensor format in dataset.
        """
        
        # Get item special method
        first_val = int(list(self.dataset_numbers.values())[0])
        if index < first_val:
            class_val = 'covid'
            label = torch.Tensor([1, 0])
        else:
            class_val = 'non_covid'
            index = index - first_val
            label = torch.Tensor([0, 1])

        im = self.open_img(self.groups, class_val, index)
        im = transforms.functional.to_tensor(np.array(im)).float()
        return im, label

In [15]:
def verify_l1_dataset(group,dataset,image_overall_index,class_val,
                   image_specific_dataset_index=1):
  """
  Helper function to verify that the classes are implemented correctly

  Parameters
  ----------
  group : str
      Allowed values: train, val, test
  dataset: object
      Object instantiated from the class
  image_overall_index: int
      Overall index in the full dataset across all classes
  class_val: str
      Image label. Example: covid, non-covid
  image_specific_dataset_index : int
      image id in the specific nested directory
  """
  print('Verify the special methods __len__ and __get_item__')
  print('Number of images in {} dataset: {}'.format(group, len(dataset)))
  print('Details for image id {} from the {} dataset'.format(
      image_overall_index,
      group
  ))
  im, class_oh = dataset[image_overall_index]
  print('Sample image shape: {}'.format(im.shape))
  print('Sample image: {}'.format(im))
  print('Sample image class: {}'.format(class_oh))

  print('\nVerify the open_img and show_img functions')
  print('Open and show image {} from the {}_{} dataset'.format(
      image_specific_dataset_index,
      group,
      class_val
  ))
  im = dataset.open_img(group, class_val, image_specific_dataset_index)
  print('Sample image shape: {}'.format(im.shape))
  print('Sample image: {}'.format(im))
  dataset.show_img(group, class_val, image_specific_dataset_index)

### 2.3.1 Layer 1 Train dataset

In [16]:
train_group = 'train'
l1_ld_train = L1_Lung_Dataset(groups = train_group, 
                              dataset_numbers = binary_dataset_numbers['layer_1'][train_group], 
                              dataset_paths = binary_dataset_paths['layer_1'][train_group])
l1_ld_train.describe()

This is the train dataset of the Lung Dataset used for the Small Project Demo in the 50.039 Deep Learning class in March 2021. 
It contains a total of 3875 images, of size 150 by 150.
The images are stored in the following locations and each one contains the following number of images:
 - train_covid, in folder ./dataset/train/infected/covid: 1345 images.
 - train_non_covid, in folder ./dataset/train/infected/non-covid: 2530 images.



### 2.3.2 Layer 1 Validation dataset

In [18]:
val_group = 'val'
l1_ld_val = L1_Lung_Dataset(groups = val_group, 
                              dataset_numbers = binary_dataset_numbers['layer_1'][val_group], 
                              dataset_paths = binary_dataset_paths['layer_1'][val_group])
l1_ld_val.describe()

This is the val dataset of the Lung Dataset used for the Small Project Demo in the 50.039 Deep Learning class in March 2021. 
It contains a total of 16 images, of size 150 by 150.
The images are stored in the following locations and each one contains the following number of images:
 - val_covid, in folder ./dataset/val/infected/covid: 8 images.
 - val_non_covid, in folder ./dataset/val/infected/non-covid: 8 images.



### 2.3.3 Layer 1 Test dataset

In [20]:
test_group = 'test'
l1_ld_test = L1_Lung_Dataset(groups = test_group, 
                              dataset_numbers = binary_dataset_numbers['layer_1'][test_group], 
                              dataset_paths = binary_dataset_paths['layer_1'][test_group])
l1_ld_test.describe()

This is the test dataset of the Lung Dataset used for the Small Project Demo in the 50.039 Deep Learning class in March 2021. 
It contains a total of 380 images, of size 150 by 150.
The images are stored in the following locations and each one contains the following number of images:
 - test_covid, in folder ./dataset/test/infected/covid: 138 images.
 - test_non_covid, in folder ./dataset/test/infected/non-covid: 242 images.



# 3. Creating a data loader object

## 3.1 Layer 0

In [27]:
l0_bs_train = 32
l0_bs_test = 32
l0_bs_val = 1
l0_train_loader = DataLoader(l0_ld_train, batch_size = l0_bs_train, shuffle = True)
l0_test_loader = DataLoader(l0_ld_test, batch_size = l0_bs_test, shuffle = True)
l0_val_loader = DataLoader(l0_ld_val, batch_size = l0_bs_val, shuffle = True)

## 3.2 Layer 1

In [28]:
l1_bs_train = 32
l1_bs_test = 32
l1_bs_val = 1
l1_train_loader = DataLoader(l1_ld_train, batch_size = l1_bs_train, shuffle = True)
l1_test_loader = DataLoader(l1_ld_test, batch_size = l1_bs_test, shuffle = True)
l1_val_loader = DataLoader(l1_ld_val, batch_size = l1_bs_val, shuffle = True)

# 4. Model

https://link.springer.com/article/10.1007/s12652-021-02917-3#Sec9



## 4.1 Layer 0

In [29]:
class L0_Net(nn.Module):
    def __init__(self, num_layers=1):
        super(L0_Net, self).__init__()
        # Conv2D: 1 input channel, 4 output channels, 3 by 3 kernel, stride of 1.
        self.conv1 = nn.Conv2d(1, 16, 3, 1)
        self.dropout1 = nn.Dropout2d(0.2)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(16, 8, 3, 1)
        self.conv3 = nn.Conv2d(8, 32, 3, 1)
        self.avgpool = nn.AvgPool2d(3)
        self.maxpool = nn.MaxPool2d(2, stride=2)
        self.fc1 = nn.Linear(16928, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.dropout1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.dropout1(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.avgpool(x)
        x = self.dropout1(x)
        x = self.relu(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        output = F.log_softmax(x, dim = 1)
        return output

In [30]:
# Activate gpu
if torch.cuda.is_available():  
    print('using GPU')
    device = "cuda:0" 
else:  
    device = "cpu"
l0_model = L0_Net().to(torch.device(device))

using GPU


In [31]:
summary(l0_model, (1, 150, 150))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 148, 148]             160
         Dropout2d-2         [-1, 16, 148, 148]               0
              ReLU-3         [-1, 16, 148, 148]               0
            Conv2d-4          [-1, 8, 146, 146]           1,160
              ReLU-5          [-1, 8, 146, 146]               0
         MaxPool2d-6            [-1, 8, 73, 73]               0
         Dropout2d-7            [-1, 8, 73, 73]               0
            Conv2d-8           [-1, 32, 71, 71]           2,336
              ReLU-9           [-1, 32, 71, 71]               0
        AvgPool2d-10           [-1, 32, 23, 23]               0
        Dropout2d-11           [-1, 32, 23, 23]               0
             ReLU-12           [-1, 32, 23, 23]               0
           Linear-13                    [-1, 2]          33,858
Total params: 37,514
Trainable params: 

## 4.2 Layer 1

In [32]:
class L1_Net(nn.Module):
    def __init__(self, num_layers=1):
        super(L1_Net, self).__init__()
        # Conv2D: 1 input channel, 4 output channels, 3 by 3 kernel, stride of 1.
        self.conv1 = nn.Conv2d(1, 16, 3, 1)
        self.dropout1 = nn.Dropout2d(0.2)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(16, 8, 3, 1)
        self.conv3 = nn.Conv2d(8, 32, 3, 1)
        self.avgpool = nn.AvgPool2d(3)
        self.maxpool = nn.MaxPool2d(2, stride=2)
        self.fc1 = nn.Linear(16928, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.dropout1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.dropout1(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.avgpool(x)
        x = self.dropout1(x)
        x = self.relu(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        output = F.log_softmax(x, dim = 1)
        return output

In [33]:
# Activate gpu
if torch.cuda.is_available():  
    print('using GPU')
    device = "cuda:0" 
else:  
    device = "cpu"
l1_model = L1_Net().to(torch.device(device))

using GPU


In [34]:
summary(l1_model, (1, 150, 150))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 148, 148]             160
         Dropout2d-2         [-1, 16, 148, 148]               0
              ReLU-3         [-1, 16, 148, 148]               0
            Conv2d-4          [-1, 8, 146, 146]           1,160
              ReLU-5          [-1, 8, 146, 146]               0
         MaxPool2d-6            [-1, 8, 73, 73]               0
         Dropout2d-7            [-1, 8, 73, 73]               0
            Conv2d-8           [-1, 32, 71, 71]           2,336
              ReLU-9           [-1, 32, 71, 71]               0
        AvgPool2d-10           [-1, 32, 23, 23]               0
        Dropout2d-11           [-1, 32, 23, 23]               0
             ReLU-12           [-1, 32, 23, 23]               0
           Linear-13                    [-1, 2]          33,858
Total params: 37,514
Trainable params: 

# 5. Training the model

Reference material: [Towards data science: PyTorch [Tabular] — Multiclass Classification](https://towardsdatascience.com/pytorch-tabular-multiclass-classification-9f8211a123ab)

In [35]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    
    return acc

# 5.1.1 GridSearch for Layer 0

## Hyperparameters 
'epochs': [5, 10]
'lr': [0.0001, 0.00001]
'scheduler_gamma': [0.1, 0.001]
'l1_lambda': [0.5, 0.01]

In [69]:
def train_model_L0(epochs, lr, scheduler_gamma, l1_lambda, log_path='../layer_0_gridsearch.log'):
    
    # Activate gpu
    if torch.cuda.is_available():  
        print('using GPU')
        device = "cuda:0" 
    else:  
        device = "cpu"
        
    # Load a fresh network
    l0_model = L0_Net().to(torch.device(device))
    
    # setup
    weight_decay = 0.00001
    l0_class_weights = torch.tensor([2.83, 1.0]).to(torch.device(device))
    criterion = nn.CrossEntropyLoss(l0_class_weights)
    optimizer = optim.AdamW(l0_model.parameters(), lr = lr, weight_decay=weight_decay)
    
    scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=scheduler_gamma)
    
    start = time.time()
    start_model_time = datetime.now().strftime("%d_%m_%Y_%H_%M_%S")
    
    L1_reg = torch.tensor(0., requires_grad=True)
    for name, param in l0_model.named_parameters():
        if 'weight' in name:
            L1_reg = L1_reg + torch.norm(param, 1)
    
    # Save log for hyperparameters
    with open(log_path, "a") as f:
        f.write("{} : Training L0 model - epochs {} - lr {} - scheduler_gamma {} - l1_lambda - {} \n".format(start_model_time, epochs, lr, scheduler_gamma, l1_lambda))
    
    l0_accuracy_stats_epoch = {
        'train': [],
        'test': [],
        'epoch': [],
    }
    l0_loss_stats_epoch = {
        'train': [],
        'test': [],
        'epoch': [],
    }

    for e in range(epochs):
        train_epoch_loss = 0
        train_epoch_acc = 0

        l0_model.train()
        for batch_idx, (X_train_batch, y_train_batch) in enumerate(l0_train_loader):
            X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)

            optimizer.zero_grad()

            output = l0_model.forward(X_train_batch)
            train_loss  = criterion(output, torch.max(y_train_batch, 1)[1])
            train_acc = multi_acc(output, torch.max(y_train_batch, 1)[1])

            train_loss.backward()
            optimizer.step()
            # scheduler.step(e + batch_idx / len(l0_train_loader))

            train_epoch_loss += train_loss.item()
            train_epoch_acc += train_acc.item()
            
        # update scheduler
        scheduler.step()

        # testing
        with torch.no_grad():
            test_epoch_loss = 0
            test_epoch_acc = 0
            l0_model.eval()
            for X_test_batch, y_test_batch in l0_test_loader:
                X_test_batch, y_test_batch = X_test_batch.to(device), y_test_batch.to(device)

                y_test_pred = l0_model.forward(X_test_batch)

                test_loss = criterion(y_test_pred, torch.max(y_test_batch, 1)[1])
                test_acc = multi_acc(y_test_pred, torch.max(y_test_batch, 1)[1])

                test_epoch_loss += test_loss.item()
                test_epoch_acc += test_acc.item()


        # averaged
        train_epoch_loss = train_epoch_loss/len(l0_train_loader)
        train_epoch_acc = train_epoch_acc/len(l0_train_loader)
        test_epoch_loss = test_epoch_loss/len(l0_test_loader)
        test_epoch_acc = test_epoch_acc/len(l0_test_loader)

        # The step number corresponds to the number of batches seen
        now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
        print("Epoch: {}/{} - {} - ".format(e+1, epochs, now),
          "Training Loss: {:.4f} - ".format(train_epoch_loss),
          "Training Accuracy: {:.4f}".format(train_epoch_acc),
          "Test Loss: {:.4f} - ".format(test_epoch_loss),
          "Test Accuracy: {:.4f}".format(test_epoch_acc))
        l0_model.train()
        
        # save the logs
        with open(log_path, "a") as f:
            f.write("Epoch: {}/{} - {} - ".format(e+1, epochs, now))
            f.write("Training Loss: {:.4f} - ".format(train_epoch_loss))
            f.write("Training Accuracy: {:.4f} - ".format(train_epoch_acc))
            f.write("Test Loss: {:.4f} - ".format(test_epoch_loss))
            f.write("Test Accuracy: {:.4f}\n\n".format(test_epoch_acc))

In [70]:
# epochs
train_model_L0(epochs= 5, lr=0.0001, scheduler_gamma=0.1, l1_lambda=0.5)
train_model_L0(epochs= 10, lr=0.0001, scheduler_gamma=0.1, l1_lambda=0.5)

# learning rate
train_model_L0(epochs= 5, lr=0.00001, scheduler_gamma=0.1, l1_lambda=0.5)
train_model_L0(epochs= 10, lr=0.00001, scheduler_gamma=0.1, l1_lambda=0.5)

# scheduler gamma
train_model_L0(epochs= 5, lr=0.0001, scheduler_gamma=0.001, l1_lambda=0.5)
train_model_L0(epochs= 10, lr=0.0001, scheduler_gamma=0.001, l1_lambda=0.5)
train_model_L0(epochs= 5, lr=0.00001, scheduler_gamma=0.001, l1_lambda=0.5)
train_model_L0(epochs= 10, lr=0.00001, scheduler_gamma=0.001, l1_lambda=0.5)

# l1 lambda
train_model_L0(epochs= 5, lr=0.0001, scheduler_gamma=0.1, l1_lambda=0.01)
train_model_L0(epochs= 10, lr=0.0001, scheduler_gamma=0.1, l1_lambda=0.01)
train_model_L0(epochs= 5, lr=0.00001, scheduler_gamma=0.1, l1_lambda=0.01)
train_model_L0(epochs= 10, lr=0.00001, scheduler_gamma=0.1, l1_lambda=0.01)
train_model_L0(epochs= 5, lr=0.0001, scheduler_gamma=0.001, l1_lambda=0.01)
train_model_L0(epochs= 10, lr=0.0001, scheduler_gamma=0.001, l1_lambda=0.01)
train_model_L0(epochs= 5, lr=0.00001, scheduler_gamma=0.001, l1_lambda=0.01)
train_model_L0(epochs= 10, lr=0.00001, scheduler_gamma=0.001, l1_lambda=0.01)

using GPU
Epoch: 1/5 - 21/03/2021 00:01:37 -  Training Loss: 0.6498 -  Training Accuracy: 0.6595 Test Loss: 0.4764 -  Test Accuracy: 0.7833
Epoch: 2/5 - 21/03/2021 00:01:45 -  Training Loss: 0.3294 -  Training Accuracy: 0.8704 Test Loss: 0.3467 -  Test Accuracy: 0.8354
Epoch: 3/5 - 21/03/2021 00:01:54 -  Training Loss: 0.2265 -  Training Accuracy: 0.9156 Test Loss: 0.7920 -  Test Accuracy: 0.7745
Epoch: 4/5 - 21/03/2021 00:02:03 -  Training Loss: 0.2008 -  Training Accuracy: 0.9291 Test Loss: 0.7452 -  Test Accuracy: 0.8031
Epoch: 5/5 - 21/03/2021 00:02:11 -  Training Loss: 0.1885 -  Training Accuracy: 0.9298 Test Loss: 0.7357 -  Test Accuracy: 0.8010
using GPU
Epoch: 1/10 - 21/03/2021 00:02:20 -  Training Loss: 0.5777 -  Training Accuracy: 0.7485 Test Loss: 0.5365 -  Test Accuracy: 0.7896
Epoch: 2/10 - 21/03/2021 00:02:29 -  Training Loss: 0.3142 -  Training Accuracy: 0.8769 Test Loss: 0.5446 -  Test Accuracy: 0.8115
Epoch: 3/10 - 21/03/2021 00:02:38 -  Training Loss: 0.2304 -  Traini

Epoch: 3/5 - 21/03/2021 00:11:00 -  Training Loss: 0.2093 -  Training Accuracy: 0.9197 Test Loss: 0.6034 -  Test Accuracy: 0.8042
Epoch: 4/5 - 21/03/2021 00:11:10 -  Training Loss: 0.1833 -  Training Accuracy: 0.9317 Test Loss: 0.7927 -  Test Accuracy: 0.7854
Epoch: 5/5 - 21/03/2021 00:11:19 -  Training Loss: 0.1786 -  Training Accuracy: 0.9344 Test Loss: 1.0744 -  Test Accuracy: 0.7641
using GPU
Epoch: 1/10 - 21/03/2021 00:11:29 -  Training Loss: 0.6811 -  Training Accuracy: 0.6081 Test Loss: 0.6301 -  Test Accuracy: 0.7651
Epoch: 2/10 - 21/03/2021 00:11:38 -  Training Loss: 0.4391 -  Training Accuracy: 0.8322 Test Loss: 0.3757 -  Test Accuracy: 0.8286
Epoch: 3/10 - 21/03/2021 00:11:47 -  Training Loss: 0.2621 -  Training Accuracy: 0.8999 Test Loss: 0.3713 -  Test Accuracy: 0.8385
Epoch: 4/10 - 21/03/2021 00:11:56 -  Training Loss: 0.2147 -  Training Accuracy: 0.9197 Test Loss: 0.6521 -  Test Accuracy: 0.8063
Epoch: 5/10 - 21/03/2021 00:12:05 -  Training Loss: 0.2076 -  Training Accur

## 5.2 Layer 1

# 5.2.1 GridSearch for Layer 1
hyperparameters
'epochs': [5, 10] 'lr': [0.0001, 0.00001] 'scheduler_gamma': [0.1, 0.001] 'l1_lambda': [0.5, 0.01]

In [72]:
def train_model_L1(epochs, lr, scheduler_gamma, l1_lambda, log_path='../layer_1_gridsearch.log'):
    
    # Activate gpu
    if torch.cuda.is_available():  
        print('using GPU')
        device = "cuda:0" 
    else:  
        device = "cpu"
        
    # Load a fresh network
    l1_model = L1_Net().to(torch.device(device))
    
    # setup
    weight_decay = 0.00001
    l1_class_weights = torch.tensor([2.83, 1.0]).to(torch.device(device))
    criterion = nn.CrossEntropyLoss(l0_class_weights)
    optimizer = optim.AdamW(l1_model.parameters(), lr = lr, weight_decay=weight_decay)
    
    scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=scheduler_gamma)
    
    start = time.time()
    start_model_time = datetime.now().strftime("%d_%m_%Y_%H_%M_%S")
    
    L1_reg = torch.tensor(0., requires_grad=True)
    for name, param in l1_model.named_parameters():
        if 'weight' in name:
            L1_reg = L1_reg + torch.norm(param, 1)
    
    # Save log for hyperparameters
    with open(log_path, "a") as f:
        f.write("{} : Training L1 model - epochs {} - lr {} - scheduler_gamma {} - l1_lambda - {} \n".format(start_model_time, epochs, lr, scheduler_gamma, l1_lambda))
    
    l1_accuracy_stats_epoch = {
        'train': [],
        'test': [],
        'epoch': [],
    }
    l1_loss_stats_epoch = {
        'train': [],
        'test': [],
        'epoch': [],
    }

    for e in range(epochs):
        train_epoch_loss = 0
        train_epoch_acc = 0

        l1_model.train()
        for batch_idx, (X_train_batch, y_train_batch) in enumerate(l1_train_loader):
            X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)

            optimizer.zero_grad()

            output = l1_model.forward(X_train_batch)
            train_loss  = criterion(output, torch.max(y_train_batch, 1)[1])
            train_acc = multi_acc(output, torch.max(y_train_batch, 1)[1])

            train_loss.backward()
            optimizer.step()

            train_epoch_loss += train_loss.item()
            train_epoch_acc += train_acc.item()
            
        # update scheduler
        scheduler.step()

        # testing
        with torch.no_grad():
            test_epoch_loss = 0
            test_epoch_acc = 0
            l1_model.eval()
            for X_test_batch, y_test_batch in l1_test_loader:
                X_test_batch, y_test_batch = X_test_batch.to(device), y_test_batch.to(device)

                y_test_pred = l1_model.forward(X_test_batch)

                test_loss = criterion(y_test_pred, torch.max(y_test_batch, 1)[1])
                test_acc = multi_acc(y_test_pred, torch.max(y_test_batch, 1)[1])

                test_epoch_loss += test_loss.item()
                test_epoch_acc += test_acc.item()


        # averaged
        train_epoch_loss = train_epoch_loss/len(l1_train_loader)
        train_epoch_acc = train_epoch_acc/len(l1_train_loader)
        test_epoch_loss = test_epoch_loss/len(l1_test_loader)
        test_epoch_acc = test_epoch_acc/len(l1_test_loader)

        # The step number corresponds to the number of batches seen
        now = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
        print("Epoch: {}/{} - {} - ".format(e+1, epochs, now),
          "Training Loss: {:.4f} - ".format(train_epoch_loss),
          "Training Accuracy: {:.4f}".format(train_epoch_acc),
          "Test Loss: {:.4f} - ".format(test_epoch_loss),
          "Test Accuracy: {:.4f}".format(test_epoch_acc))
        l1_model.train()

        # Epoch metrics
        l1_loss_stats_epoch['train'].append(train_epoch_loss)
        l1_loss_stats_epoch['test'].append(test_epoch_loss)
        l1_loss_stats_epoch['epoch'].append(e+1)
        l1_accuracy_stats_epoch['train'].append(train_epoch_acc)
        l1_accuracy_stats_epoch['test'].append(test_epoch_acc)
        l1_accuracy_stats_epoch['epoch'].append(e+1)

        # save the logs
        with open(log_path, "a") as f:
            f.write("Epoch: {}/{} - {} - ".format(e+1, epochs, now))
            f.write("Training Loss: {:.4f} - ".format(train_epoch_loss))
            f.write("Training Accuracy: {:.4f} - ".format(train_epoch_acc))
            f.write("Test Loss: {:.4f} - ".format(test_epoch_loss))
            f.write("Test Accuracy: {:.4f}\\n".format(test_epoch_acc))

In [73]:
# epochs
train_model_L1(epochs= 5, lr=0.0001, scheduler_gamma=0.1, l1_lambda=0.5)
train_model_L1(epochs= 10, lr=0.0001, scheduler_gamma=0.1, l1_lambda=0.5)

# learning rate
train_model_L1(epochs= 5, lr=0.00001, scheduler_gamma=0.1, l1_lambda=0.5)
train_model_L1(epochs= 10, lr=0.00001, scheduler_gamma=0.1, l1_lambda=0.5)

# scheduler gamma
train_model_L1(epochs= 5, lr=0.0001, scheduler_gamma=0.001, l1_lambda=0.5)
train_model_L1(epochs= 10, lr=0.0001, scheduler_gamma=0.001, l1_lambda=0.5)
train_model_L1(epochs= 5, lr=0.00001, scheduler_gamma=0.001, l1_lambda=0.5)
train_model_L1(epochs= 10, lr=0.00001, scheduler_gamma=0.001, l1_lambda=0.5)

# l1 lambda
train_model_L1(epochs= 5, lr=0.0001, scheduler_gamma=0.1, l1_lambda=0.01)
train_model_L1(epochs= 10, lr=0.0001, scheduler_gamma=0.1, l1_lambda=0.01)
train_model_L1(epochs= 5, lr=0.00001, scheduler_gamma=0.1, l1_lambda=0.01)
train_model_L1(epochs= 10, lr=0.00001, scheduler_gamma=0.1, l1_lambda=0.01)
train_model_L1(epochs= 5, lr=0.0001, scheduler_gamma=0.001, l1_lambda=0.01)
train_model_L1(epochs= 10, lr=0.0001, scheduler_gamma=0.001, l1_lambda=0.01)
train_model_L1(epochs= 5, lr=0.00001, scheduler_gamma=0.001, l1_lambda=0.01)
train_model_L1(epochs= 10, lr=0.00001, scheduler_gamma=0.001, l1_lambda=0.01)

using GPU
Epoch: 1/5 - 21/03/2021 00:39:40 -  Training Loss: 0.6748 -  Training Accuracy: 0.3698 Test Loss: 0.6437 -  Test Accuracy: 0.3638
Epoch: 2/5 - 21/03/2021 00:39:47 -  Training Loss: 0.6573 -  Training Accuracy: 0.3834 Test Loss: 0.5966 -  Test Accuracy: 0.4364
Epoch: 3/5 - 21/03/2021 00:39:53 -  Training Loss: 0.6383 -  Training Accuracy: 0.4481 Test Loss: 0.5407 -  Test Accuracy: 0.6034
Epoch: 4/5 - 21/03/2021 00:39:59 -  Training Loss: 0.6184 -  Training Accuracy: 0.5396 Test Loss: 0.5024 -  Test Accuracy: 0.6343
Epoch: 5/5 - 21/03/2021 00:40:05 -  Training Loss: 0.6095 -  Training Accuracy: 0.5700 Test Loss: 0.5018 -  Test Accuracy: 0.8110
using GPU
Epoch: 1/10 - 21/03/2021 00:40:11 -  Training Loss: 0.6742 -  Training Accuracy: 0.3539 Test Loss: 0.6475 -  Test Accuracy: 0.3631
Epoch: 2/10 - 21/03/2021 00:40:17 -  Training Loss: 0.6629 -  Training Accuracy: 0.3813 Test Loss: 0.5964 -  Test Accuracy: 0.3769
Epoch: 3/10 - 21/03/2021 00:40:23 -  Training Loss: 0.6336 -  Traini

Epoch: 3/5 - 21/03/2021 00:46:28 -  Training Loss: 0.6508 -  Training Accuracy: 0.4035 Test Loss: 0.5964 -  Test Accuracy: 0.7679
Epoch: 4/5 - 21/03/2021 00:46:35 -  Training Loss: 0.6364 -  Training Accuracy: 0.4987 Test Loss: 0.5168 -  Test Accuracy: 0.6276
Epoch: 5/5 - 21/03/2021 00:46:42 -  Training Loss: 0.6269 -  Training Accuracy: 0.5396 Test Loss: 0.5056 -  Test Accuracy: 0.6949
using GPU
Epoch: 1/10 - 21/03/2021 00:46:49 -  Training Loss: 0.6746 -  Training Accuracy: 0.3570 Test Loss: 0.6432 -  Test Accuracy: 0.3635
Epoch: 2/10 - 21/03/2021 00:46:56 -  Training Loss: 0.6528 -  Training Accuracy: 0.4219 Test Loss: 0.5827 -  Test Accuracy: 0.5848
Epoch: 3/10 - 21/03/2021 00:47:03 -  Training Loss: 0.6302 -  Training Accuracy: 0.4899 Test Loss: 0.5314 -  Test Accuracy: 0.7872
Epoch: 4/10 - 21/03/2021 00:47:10 -  Training Loss: 0.6105 -  Training Accuracy: 0.5616 Test Loss: 0.4853 -  Test Accuracy: 0.7928
Epoch: 5/10 - 21/03/2021 00:47:16 -  Training Loss: 0.6014 -  Training Accur